# IMPORT DATA, PACKAGES, AND FUNCTIONS

## import the relevant functions and packages

In [1]:
import pandas as pd
import numpy as np
from test_case_generator_functions import alphabet

# TEST CASE TYPE REFERENCE INFORMATION

In [2]:
uid = 'UID-255' 
theme = 'Names where name parts are Modified'
category = 'Transpositions'
sub_category = '2 transpositions (different name part)'
entity_type = 'Entity'

## download the OFAC list from the web

In [3]:
ofac_list_download = pd.read_csv('https://www.treasury.gov/ofac/downloads/sdn.csv', header=None)

In [4]:
ofac_list = ofac_list_download[[0,1,2]]
ofac_list.columns = ['uid', 'name', 'entity_type']

# FILTER FOR THE REQUIREMENTS OF THE TEST CASE TYPE

## filter for the requirements of the specific test case type

In [5]:
ofac_list_filtered = ofac_list[(ofac_list.entity_type == '-0- ')] # only evaluate entities

## randomly choose 10 rows

In [6]:
while True:
    ofac_list_sampled = ofac_list_filtered.sample(n = 10)
    cnt = [0] * 10
    for ind, name in enumerate(ofac_list_sampled['name']):
        split_name = name.upper().split(' ')
        
        for i in split_name:
            if all(j not in alphabet() for j in i):
                split_name.remove(i) # remove word with all numbers
            elif len(i) == 1:
                split_name.remove(i) # remove word with only 1 letter
                
        cnt[ind] += len(split_name)

    if all(i >= 2 for i in cnt):
        break

ofac_list_sampled

,uid,name,entity_type
1381,10253,OVERSEAS TRADING COMPANY,-0-
3100,15697,SYNERGY GENERAL TRADING FZE,-0-
6873,26513,PAO ZVEZDA,-0-
5261,22416,"IFDK, ZAO",-0-
1888,11488,FRANZUL S.A.,-0-
3793,17243,INDUSTRIAL BANK,-0-
3085,15673,"GRUPO COMERCIAL ROOL, S.A. DE C.V.",-0-
2669,13356,"I&S HOLDING COMPANY, S.A.",-0-
9967,36359,JOINT STOCK COMPANY MARSHAL.GLOBAL,-0-
5629,23463,IRAN HELICOPTER SUPPORT AND RENEWAL COMPANY,-0-


# CREATE THE TEST CASES

## create blank final test cases table

In [7]:
final_test_cases = pd.DataFrame(columns=['UID', 'Theme','Category','Sub-category','Entity-Type','Test Case ID' , 'OFAC List UID', 'Original Name','Test Case Name'])
final_test_cases

,UID,Theme,Category,Sub-category,Entity-Type,Test Case ID,OFAC List UID,Original Name,Test Case Name


## run loop to generate the test cases

In [8]:
for index, row in ofac_list_sampled.iterrows():
    original_name = row['name'].upper()
    split_original_name = original_name.split(' ') # split name into word
    final_test_name = split_original_name.copy()
    
    for i in split_original_name:
        if all(j not in alphabet() for j in i):
            split_original_name.remove(i) # remove word with all numbers
        elif len(i) == 1:
            split_original_name.remove(i) # remove word with only 1 letter
        
    for i in range(2):
        replace_word = np.random.choice(split_original_name) # randomly choose word to be transposed
        split_original_name.remove(replace_word) # remove chosen word
        
        random_list = list(range(len(replace_word))) # randomly choose letter to be transposed
        while True:
            random_letter_index = np.random.choice(random_list)
            random_letter = replace_word[random_letter_index]
            if random_letter in alphabet():
                break
        random_list.remove(random_letter_index) # remove chosen letter index
        
        random_place = np.random.choice(random_list) # randomly choose place to transpose to
        
        replace_word_list = list(replace_word)
        del replace_word_list[random_letter_index] # remove chosen letter
        replace_word_list.insert(random_place, random_letter) # add transposed letter
        
        final_test_name[final_test_name.index(replace_word)] = ''.join(replace_word_list)

    final_test_name = ' '.join(final_test_name)
    
    final_test_cases.loc[len(final_test_cases)] = [uid, theme, category, sub_category, entity_type, uid + ' - ' + str(index), row['uid'], row['name'], final_test_name] # append to the dataframe

final_test_cases

,UID,Theme,Category,Sub-category,Entity-Type,Test Case ID,OFAC List UID,Original Name,Test Case Name
0,UID-255,Names where name parts are Modified,Transpositions,2 transpositions (different name part),Entity,UID-255 - 1381,10253,OVERSEAS TRADING COMPANY,VEROSEAS TRADNIG COMPANY
1,UID-255,Names where name parts are Modified,Transpositions,2 transpositions (different name part),Entity,UID-255 - 3100,15697,SYNERGY GENERAL TRADING FZE,SYNERGY GENERAL TRNADIG FEZ
2,UID-255,Names where name parts are Modified,Transpositions,2 transpositions (different name part),Entity,UID-255 - 6873,26513,PAO ZVEZDA,AOP ZEVZDA
3,UID-255,Names where name parts are Modified,Transpositions,2 transpositions (different name part),Entity,UID-255 - 5261,22416,"IFDK, ZAO","FIDK, AZO"
4,UID-255,Names where name parts are Modified,Transpositions,2 transpositions (different name part),Entity,UID-255 - 1888,11488,FRANZUL S.A.,FRAZUNL .SA.
5,UID-255,Names where name parts are Modified,Transpositions,2 transpositions (different name part),Entity,UID-255 - 3793,17243,INDUSTRIAL BANK,INTDUSRIAL KBAN
6,UID-255,Names where name parts are Modified,Transpositions,2 transpositions (different name part),Entity,UID-255 - 3085,15673,"GRUPO COMERCIAL ROOL, S.A. DE C.V.","GRUOP COMECIALR ROOL, S.A. DE C.V."
7,UID-255,Names where name parts are Modified,Transpositions,2 transpositions (different name part),Entity,UID-255 - 2669,13356,"I&S HOLDING COMPANY, S.A.","I&S HOLDING CMPANYO, .AS."
8,UID-255,Names where name parts are Modified,Transpositions,2 transpositions (different name part),Entity,UID-255 - 9967,36359,JOINT STOCK COMPANY MARSHAL.GLOBAL,JOINT OSTCK COMPANY MARSLHAL.GLOBA
9,UID-255,Names where name parts are Modified,Transpositions,2 transpositions (different name part),Entity,UID-255 - 5629,23463,IRAN HELICOPTER SUPPORT AND RENEWAL COMPANY,IRAN HELICOPTER SPUPORT AND ERNEWAL COMPANY
